# 02 — Preprocesamiento EEG de Imaginacion Motora (4 clases)

Este notebook prepara un **dataset limpio** de EEG para clasificacion **multiclase (4C)** con las clases:  
**Left / Right / Both Fists / Both Feet** utilizando **solo imagineria motora (MI)** del dataset PhysioNet EEG Motor Movement/Imagery.

## Objetivo
- Leer los EDF de los runs de MI.
- Normalizar y limpiar senales (nombres de canales, notch, referencia CAR, interpolacion, ICA opcional).
- Filtrar en banda **8-30 Hz** (mu/beta).
- Epocar ventanas **0.5-4.5 s** post-onset por clase.
- Rechazar ensayos con artefactos.
- Guardar **Epochs por sujeto** listos para entrenar modelos.

## Salidas
- Por sujeto: `data/processed/S###_MI-epo.fif`
- Resumen por sujeto: `reports/tables/EDA_raw/preproc_epochs_count_per_subject.csv`

> Las secciones estan divididas por bloques con titulo y explicacion.


## Rutas del proyecto y estructura de carpetas

Definimos las rutas del proyecto segun tu estructura (el notebook vive en `notebooks/`).  
Creamos (si no existen) las carpetas para salidas intermedias y reportes.


In [ ]:
# [1] Rutas del proyecto
from pathlib import Path

# Usamos la raiz del repo subiendo un nivel desde notebooks/
PROJ = Path('..').resolve()

# Estructura
DATA_RAW = PROJ / 'data' / 'raw'             # EDF crudos
DATA_PROC = PROJ / 'data' / 'processed'      # epochs limpios listos para modelos

# Directorios de reportes/figuras/tablas para este notebook
REPORT_FIG = PROJ / 'reports' / 'figures' / '02_prepro'
REPORT_TAB = PROJ / 'reports' / 'tables' / '02_prepro'

# Crear carpetas si no existen
for p in [DATA_PROC, REPORT_FIG, REPORT_TAB]:
    p.mkdir(parents=True, exist_ok=True)

# Carpeta de resúmenes por sujeto/run
SUBJ_SUMMARY_DIR = DATA_PROC / 'subjectssummary'
SUBJ_SUMMARY_DIR.mkdir(parents=True, exist_ok=True)

print('PROJ:', PROJ)
print('DATA_RAW existe?:', DATA_RAW.exists(), DATA_RAW)
print('REPORT_FIG:', REPORT_FIG)
print('REPORT_TAB:', REPORT_TAB)


PROJ: C:\Users\joelc\Desktop\eeg2
DATA_RAW existe?: True C:\Users\joelc\Desktop\eeg2\data\raw
REPORT_FIG: C:\Users\joelc\Desktop\eeg2\reports\figures\02_prepro
REPORT_TAB: C:\Users\joelc\Desktop\eeg2\reports\tables\02_prepro


Define la raíz del repo (PROJ) y estandariza carpetas:

- data/raw: archivos EDF crudos (PhysioNet).
- data/processed: epochs limpios en formato FIF listos para modelos.
- reports/figures y reports/tables para guardar salidas de QA/diagnóstico.
- data/processed/subjectssummary para reportes por sujeto y consolidado.

Crea carpetas si no existen y muestra “sanity checks”.

## Dependencias y parametros globales

Definimos librerias, constantes de preprocesamiento, runs a usar, ventana de epoching y umbrales de rechazo.

**Por que estos parametros**:
- **EXCLUDE_SUBJECTS**: sujetos con anotaciones problematicas reportados en estudios previos.
- **MI_RUNS_LR/MI_RUNS_OF**: runs de imagineria (no movimiento real) para manos (Left/Right) y punos/pies (O/F).
- **TMIN/TMAX = 0.5-4.5 s**: evitamos el onset inmediato (transitorios/artefactos) y capturamos ERD/ERS.
- **Notch 60/120 Hz**: elimina red electrica y su armonico.
- **Banda 8-30 Hz**: bandas mu/beta asociadas a MI.
- **ICA**: quitar parpadeos (EOG) y algo de EMG si es posible.
- **Umbrales de rechazo**: evitar ensayos con amplitudes atipicas (artefactos).


In [ ]:
# [2] Dependencias y parametros globales
import re, numpy as np, pandas as pd, mne
from IPython.display import display

# Seleccion de sujetos — Excluir sujetos con problemas de anotacion
EXCLUDE_SUBJECTS = {38, 88, 89, 92, 100, 104}

# Seleccion de runs (solo MI) — Task 2 (L/R) y Task 4 (O/F)
MI_RUNS_LR = [4, 8, 12]    # imagineria izquierda/derecha
MI_RUNS_OF = [6, 10, 14]   # imagineria ambos punos / ambos pies

# Epoching — ventana temporal y baseline
TMIN, TMAX = 0.5, 4.5
BASELINE = None  # ya filtramos; evitar baseline fuera de la ventana

# Filtrado — notch y banda (armónicos se calcularán respetando Nyquist)
NOTCH_BASE = 60.0
BP_LO, BP_HI = 8.0, 30.0

# Rechazo automatico — umbrales de amplitud
SOFTCLIP_DEFAULT = 400.0   # soft-clip global
PEAK_TO_PEAK_UV = 300.0    # uV pico a pico (por canal)
ABS_MAX_UV      = None   # uV absolutos (umbral por ensayo)

# Artefactos - ICA
RUN_ICA     = True         # True para activar ICA
ICA_METHOD  = 'auto'         # 'auto' → intenta 'picard' y si no, 'fastica'
ICA_HP, ICA_LP = 1.0, 45.0   # banda para estimar la ICA
ICA_MAX_ITER = 1000          # iters máx para fastica; picard ignora esto
ICA_TOL      = 5e-4          # tolerancia (fastica)
# - entero (ej. 32, 64) -> usa ese # de PCs
# - float en (0,1] (ej. 0.99) -> conserva ese % de varianza
ICA_N_COMPS = 32

# [Opcional] Resample para homogeneizar fs (None = dejar original, p.ej., 160 Hz)
RESAMPLE_TO = None  # 128.0

RANDOM_STATE = 42
mne.set_log_level('WARNING')


## Utilidades: parseo de nombres, normalizacion de canales y ayuda con eventos

- `parse_subject_run`: extrae `S###` y `R##` desde nombres tipo `S001R04.edf`.
- `run_kind`: clasifica un run como `LR` (Left/Right) u `OF` (Both Fists/Feet).
- `normalize_channel_names`: unifica nombres (p.ej., `C3.. -> C3`, `EEG CZ-REF -> Cz`) para trabajar con el estandar 10-20.
- `clean_event_id`: asegura que `event_id` tenga enteros para MNE.
- `list_subjects`: genera la lista de sujetos validos (excluyendo los 6 problematicos).


In [ ]:
# Utilidades: parseo y normalizacion de canales
_re_file = re.compile(r'[Ss](\d{3}).*?[Rr](\d{2})')

def parse_subject_run(path):
    """Devuelve (subject, run) a partir de una ruta, o (None, None) si no matchea."""
    m = _re_file.search(str(path))
    if not m:
        return None, None
    return int(m.group(1)), int(m.group(2))

def run_kind(run_id:int):
    """Clasifica el run como imagineria de manos (LR) o punos/pies (OF)."""
    if run_id in MI_RUNS_LR: return 'LR'
    if run_id in MI_RUNS_OF: return 'OF'
    return None

def normalize_channel_name(ch: str) -> str:
    """
    Normaliza nombres tipicos de PhysioNet/BCI2000 a estandar 10-20:
    - Quita prefijos/sufijos (EEG, -REF), puntos, guiones.
    - Convierte 'C03' -> 'C3'.
    - Ajusta 'CZ'->'Cz', 'FZ'->'Fz', 'PZ'->'Pz', etc.
    """
    s = ch.upper()
    for token in ['EEG', 'REF']:
        s = s.replace(token, '')
    import re as _re
    s = _re.sub(r'[^A-Z0-9]+', '', s)           # quita . - : espacios
    s = _re.sub(r'([A-Z])0([0-9])', r'\1\2', s) # C03->C3
    if s.endswith('Z'): s = s[:-1] + 'z'        # CZ->Cz, FZ->Fz...
    aliases = {'CZ':'Cz','FZ':'Fz','PZ':'Pz','POZ':'Poz','FCZ':'Fcz','CPZ':'Cpz','OZ':'Oz'}
    return aliases.get(s, s)

def normalize_channel_names(raw: mne.io.BaseRaw) -> None:
    """Aplica normalizacion de nombres de canal sobre el objeto Raw."""
    raw.rename_channels({ch: normalize_channel_name(ch) for ch in raw.ch_names})

def clean_event_id(event_id: dict) -> dict:
    """Asegura que event_id sea {str: int}, p.ej. {'T1': 2, 'T2': 3}."""
    out = {}
    for k, v in event_id.items():
        try:
            out[str(k).strip()] = int(v)
        except Exception:
            pass
    return out

def list_subjects():
    """Lista sujetos validos (excluye los 6 reportados)."""
    subs = []
    for sdir in sorted(DATA_RAW.glob('S*')):
        if not sdir.is_dir(): 
            continue
        try:
            subs.append(int(sdir.name[1:]))
        except:
            pass
    subs = [s for s in subs if s not in EXCLUDE_SUBJECTS]
    print(f'Sujetos validos: {len(subs)}')
    return subs


## Utilidades de saneo y detección de canales problemáticos

ensure_finite_raw: reemplaza NaN/±Inf por 0.0 en raw._data.

soft_clip_inplace: clip suave en ±500 µV antes de filtrar/ICA.

Evita que outliers extremos rompan filtros FIR/ICA (numerical overflow).

Garantiza estabilidad numérica.

In [ ]:
# Utilidad: asegurar que no haya NaN/Inf en raw._data y soft-clipping de seguridad
def ensure_finite_raw(raw, where=''):
    """
    Reemplaza NaN/+Inf/-Inf por 0.0 en raw._data (en VOLTS).
    - 'where' es un texto opcional para log (p.ej., 'tras ICA').
    """
    X = raw.get_data()  # (n_channels, n_samples)
    mask = ~np.isfinite(X)
    if mask.any():
        n_bad = int(mask.sum())
        print(f"{n_bad} valores no finitos detectados {(' ' + where) if where else ''}. Se reemplazan por 0.0")
        raw._data[:] = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    return raw

def soft_clip_inplace(raw, limit_uv=500.0):
    """
    Limita valores extremos |x|>limit_uv (en µV) ANTES de filtrar/ICA para evitar overflow/NaN.
    Modifica raw in-place.
    """
    lim_v = float(limit_uv) * 1e-6
    X = raw.get_data()
    np.clip(X, -lim_v, lim_v, out=X)
    raw._data[:] = X
    return raw


## Carga de un run (Raw EDF) y asignacion de montaje

- Cargamos el EDF con `mne.io.read_raw_edf` (preload=True).
- Normalizamos nombres de canales y asignamos montaje `standard_1020` si es posible.

Permite usar un archivo wfdbcal (si existe) para mapear nombres de canal a esquema 10–20.

edf_to_1010_name: heurística para pasar de etiquetas EDF a nombres 10–20 coherentes.

load_raw:

- lee EDF,
- selecciona solo EEG,
- normaliza nombres (con wfdbcal si hay),
- asigna montaje 10–20.

La correspondencia de canales a localización anatómica (montaje) es esencial para análisis topográficos y coherencia inter-run/sujeto.


In [ ]:
# Normalización con wfdbcal (si existe) + carga de EDF
WFDBCAL_PATH = DATA_RAW / "wfdbcal"  # ajusta ruta si fuera necesario

def edf_to_1010_name(edf_name: str) -> str:
    n = edf_name.strip()
    n = re.sub(r'[-_.]+$', '', n)
    n = re.sub(r'[-_.]?(REF|LE|A1|A2)$', '', n, flags=re.I)
    n = n.replace('-', '').replace('.', '')
    n = re.sub(r'\s+', '', n)
    u = n.upper()
    u = u.replace('FP', 'Fp')
    u = re.sub(r'([A-Za-z])Z$', r'\1z', u)
    u = re.sub(r'^T3$', 'T7', u); u = re.sub(r'^T4$', 'T8', u)
    u = u.replace('IZ', 'Iz')
    return u

def parse_wfdbcal(path: Path) -> list[str]:
    if not path.exists(): return []
    chs = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"): continue
            chs.append(line.split()[0])
    return chs

def build_mapping_from_wfdbcal(raw, wfdbcal_path):
    chs = parse_wfdbcal(wfdbcal_path); mapping = {}
    wf_set = set(chs)
    for ch in raw.ch_names:
        if ch in wf_set:
            mapping[ch] = edf_to_1010_name(ch)
        else:
            cands = [ch, ch.rstrip('.'), ch.rstrip('.').rstrip('.')]
            hit = next((c for c in cands if c in wf_set), None)
            mapping[ch] = edf_to_1010_name(hit if hit is not None else ch)
    return mapping

def normalize_channel_names_with_wfdbcal(raw):
    mapping = build_mapping_from_wfdbcal(raw, WFDBCAL_PATH) if WFDBCAL_PATH.exists() else \
              {ch: edf_to_1010_name(ch) for ch in raw.ch_names}
    mne.rename_channels(raw.info, mapping)

def load_raw(edf_path):
    """Lee EDF, selecciona EEG, normaliza nombres (wfdbcal si hay), asigna montaje."""
    raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)
    raw.pick(mne.pick_types(raw.info, eeg=True))
    normalize_channel_names_with_wfdbcal(raw)
    try:
        mont = mne.channels.make_standard_montage('standard_1020')
        raw.set_montage(mont, on_missing='ignore')
    except Exception:
        pass
    return raw


## Filtrado Notch (60/120 Hz)

Calcula armónicos válidos de 60 Hz que no excedan Nyquist.

Aplica raw.notch_filter en FIR fase zero.

Quita línea y armónicos sin distorsión de fase (filtros lineales “zero-phase”).


In [ ]:
# Notch 60 Hz y armónicos válidos según Nyquist
def _valid_notch_freqs(sfreq: float, base: float, include_harmonics: bool) -> list[float]:
    nyq = sfreq / 2.0
    if base is None or base <= 0: return []
    freqs = [base]
    if include_harmonics:
        k = 2
        while k * base < nyq - 1e-9:
            freqs.append(k * base); k += 1
    return sorted({f for f in freqs if f < nyq - 1e-9})

def apply_notch(raw):
    """Aplica notch en 60 Hz y armónicos que no excedan Nyquist."""
    freqs = _valid_notch_freqs(raw.info['sfreq'], NOTCH_BASE, include_harmonics=True)
    if freqs:
        raw.notch_filter(freqs=freqs, picks='eeg',
                         method='fir', fir_design='firwin', phase='zero', verbose=False)
    return raw


## Referencia comun promedio (CAR) e interpolacion de canales ruidosos

Detecta canales outlier por desviación estándar (>4× mediana).

Los marca como “bad” y los interpola.

Aplica referencia promedio (CAR).

Interpolar canales muy ruidosos evita contaminar CAR.

CAR reduce “bias” espacial, buen default para MI si no hay referencia preferida.


In [ ]:
# Referencia CAR e interpolacion de canales ruidosos
def apply_car_and_interpolation(raw):
    """
    Marca canales outliers por varianza y los interpola, luego aplica referencia comun promedio (CAR).
    """
    data = raw.get_data(picks='eeg') * 1e6  # uV
    ch_std = np.std(data, axis=1)
    thr = float(np.median(ch_std) * 4.0)    # heuristica 4x mediana
    bad_idx = np.where((ch_std > thr) | np.isnan(ch_std))[0].tolist()
    if bad_idx:
        raw.info['bads'] = [raw.ch_names[i] for i in bad_idx]
        raw.interpolate_bads(reset_bads=True, verbose=False)
    raw.set_eeg_reference('average', verbose=False)  # Common Average Reference
    return raw


## ICA para remover artefactos (EOG/EMG)

Estima ICA en una copia filtrada 0.5–45 Hz para captar mejor artefactos.

Elige método:

- picard si está disponible (rápido y estable),
- fallback fastica (con tol, max_iter).

Limita n_components (≤ canales − 1 y ≤ 32 por cap).

Intenta identificar componentes EOG:

- con create_eog_epochs si hay eventos, o
- utilizando canales frontales (Fp1/Fp2/AF7/AF8) como proxy.

Aplica ica.exclude al raw original (no a la copia).

ICA separa fuentes; remover EOG/EMG mejora SNR de mu/beta.

Estimar en banda amplia evita que el band-pass de 8–30 “mezcle” artefactos.


In [ ]:
# %% [8] ICA (opcional) para remover artefactos (Picard → FastICA)
from mne.preprocessing import ICA, create_eog_epochs

def _pick_ica_method():
    """
    Devuelve ('picard' | 'fastica'), respetando ICA_METHOD:
    - 'auto': usa picard si está instalado; si no, fastica.
    - 'picard'/'fastica': fuerza ese método (si falla, cae a fastica).
    """
    if ICA_METHOD.lower() in ('picard', 'fastica'):
        return ICA_METHOD.lower()

    # auto: intentamos picard
    try:
        _ = ICA(method='picard')
        return 'picard'
    except Exception:
        return 'fastica'

def apply_ica(raw: mne.io.BaseRaw):
    """
    Estima ICA en 0.5–45 Hz (copia filtrada), detecta EOG si es posible y aplica exclusiones.
    - Usa 'picard' si está disponible (pip install picard o mne[picard]); sino 'fastica'.
    - Limita n_components a min(ICA_N_COMPS_MAX, n_channels-1) para acelerar.
    - Usa decimación interna (p.ej. decim=2) para estimar más rápido.
    """
    if not RUN_ICA:
        return raw

    # Copia para estimar la ICA en banda más amplia (no alteramos el raw original aún)
    raw_ica = raw.copy().filter(ICA_HP, ICA_LP, picks='eeg', verbose=False)

    # n_components razonable para acelerar
    n_ch = len(mne.pick_types(raw_ica.info, eeg=True, eog=False))
    # n_components: entero fijo o fracción de varianza
    # - si float en (0,1], MNE conservará los primeros k PCs que explican esa varianza
    # - si entero, usará ese # de PCs (capado por rank)
    if isinstance(ICA_N_COMPS, float) and (0.0 < ICA_N_COMPS <= 1.0):
        n_components = ICA_N_COMPS
    else:
        # asegurar entero positivo y no mayor que n_ch-1 (CAR reduce un grado)
        n_components = max(1, min(int(ICA_N_COMPS), n_ch - 1))

    # Elegir método
    method = _pick_ica_method()

    # Parámetros por método
    fit_params = None
    max_iter = 'auto'
    if method == 'fastica':
        # FastICA (sklearn) admite tol y max_iter
        fit_params = dict(tol=ICA_TOL)
        max_iter = ICA_MAX_ITER

    # Instanciar ICA
    ica = ICA(
        n_components=n_components,
        method=method,
        random_state=RANDOM_STATE,
        max_iter=max_iter,
        fit_params=fit_params
    )

    # Ajuste ICA (decimación para acelerar sin perder morfología)
    ica.fit(raw_ica, picks='eeg', decim=2, verbose=False)

    # Detección de EOG automática si es posible (dos estrategias)
    try:
        # 1) Con epochs de EOG (si MNE detecta eventos)
        eog_epochs = create_eog_epochs(raw_ica, reject=None, verbose=False)
        eog_inds, _ = ica.find_bads_eog(eog_epochs, threshold=2.5)
        ica.exclude = list(set(ica.exclude) | set(eog_inds))
    except Exception:
        # 2) Con canal frontal como proxy si existe (Fp1/Fp2/AF7/AF8)
        for proxy in ('Fp2', 'Fp1', 'AF7', 'AF8'):
            if proxy in raw_ica.ch_names:
                try:
                    eog_inds, _ = ica.find_bads_eog(raw_ica, ch_name=proxy, threshold=2.5)
                    ica.exclude = list(set(ica.exclude) | set(eog_inds))
                    break
                except Exception:
                    pass

    # Aplicar ICA al raw original
    raw_clean = ica.apply(raw.copy(), exclude=ica.exclude, verbose=False)
    return raw_clean


## Filtro pasa-banda 8-30 Hz (mu/beta)

Se aplica despues de la ICA para enfatizar ritmos mu/beta asociados a MI.

Aplica FIR 8–30 Hz, fase zero-double.

Enfoca en ritmos mu (8–12) y beta (13–30), donde aparece ERD/ERS de MI.

zero-double minimiza distorsión de fase (adelanto/atraso).


In [ ]:
# Band-pass 8-30 Hz
def apply_bandpass(raw):
    """Aplica filtro pasa-banda 8-30 Hz para enfatizar ritmos mu/beta."""
    raw.filter(BP_LO, BP_HI, picks='eeg',
               method='fir', fir_design='firwin', phase='zero-double',
               verbose=False)
    return raw


## Utilidades de resumen de drops y clases

Analiza drop_log de MNE tras drop_bad.

Cuenta cuántos rechazos se asocian a cada canal (match exacto y por substring).

Formatea un “top de canales problemáticos”.

In [ ]:
# Utilidades: conteo de drops y ranking de canales
from collections import Counter

def summarize_drop_log(drop_log, total_epochs, ch_names):
    """
    Resume canales implicados en los rechazos de epochs usando drop_log.
    - Cuenta ocurrencias por canal (si el nombre aparece en la razón).
    - Si MNE no pone el nombre exacto del canal, intenta matchear por substring.
    """
    channel_counter = Counter()
    dropped_by_bad = 0

    ch_set = set(ch_names)
    ch_upper = {c.upper(): c for c in ch_names}  # mapa para recuperar casing original

    for reasons in drop_log:
        if len(reasons) == 0:
            continue  # epoch retenido

        # Intentar extraer canales:
        found_any_channel = False
        for r in reasons:
            if not isinstance(r, str):
                continue
            r_up = r.upper()

            # 1) caso simple: la razón es exactamente el nombre del canal
            if r in ch_set:
                channel_counter[r] += 1
                found_any_channel = True
                continue

            # 2) fallback: buscar si algún nombre de canal aparece como substring
            #    (p.ej. "peak_to_peak C3" o "BAD_C3")
            for cu, corig in ch_upper.items():
                if cu in r_up:
                    channel_counter[corig] += 1
                    found_any_channel = True

        if found_any_channel:
            dropped_by_bad += 1

    top = []
    if total_epochs > 0:
        for ch, cnt in channel_counter.most_common():
            pct = 100.0 * cnt / total_epochs
            top.append((ch, cnt, pct))

    return dropped_by_bad, top

def format_top_channels(top):
    if not top:
        return '—'
    return ', '.join([f"{ch}({pct:.1f}%)" for ch, cnt, pct in top[:5]])


## Fallback a .edf.event (cuando faltan T1/T2)

Si el EDF no trae anotaciones T1/T2, busca un archivo paralelo .edf.event.

Lee pares (onset_seg, code) donde code ∈ {T0,T1,T2}.

En PhysioNet hay runs con anotaciones faltantes/raras. Este fallback resuelve huecos.

In [ ]:
# Fallback para eventos desde archivo .edf.event de PhysioNet
def read_event_file_fallback(edf_path: Path):
    """
    Si no hay anotaciones T1/T2 en el EDF, intenta leer 'SxxxRyy.edf.event'
    y devuelve una lista de (onset_segundos, 'T1'|'T2'|'T0').
    """
    ev_path = Path(edf_path).with_suffix(Path(edf_path).suffix + ".event")
    if not ev_path.exists():
        return []
    rows = []
    with open(ev_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"): 
                continue
            parts = line.split()
            try:
                onset = float(parts[0]); code  = parts[1].upper()
                if code in {"T0","T1","T2"}:
                    rows.append((onset, code))
            except Exception:
                continue
    return rows


## Creacion de epochs (0.5-4.5 s) y mapeo de etiquetas a 4 clases
1. Carga y prepro en orden seguro:

    - soft_clip → notch → interp+CAR → ICA → soft_clip → band-pass → (opcional) resample.
    - Asegura finitos entre pasos (evita NaN/Inf).

2. Eventos T1/T2:

    - Normaliza etiquetas, de-duplica eventos muy pegados (<0.5 s).

    - Si faltan T1 o T2, fallback a .edf.event.

    - Mapea códigos por tipo de run:

        - LR: T1→1 (Left), T2→2 (Right).
        - OF: T1→3 (Both Fists), T2→4 (Both Feet).

3. Epoching:

    - Epochs(raw, events, event_id, tmin=0.5, tmax=4.5, baseline=None).
    - reject_by_annotation=False para no perder trials por anotaciones genéricas.

4. Rechazo de artefactos:

    - drop_bad por pico-a-pico (300 µV).
    - Ranking de canales responsables (bloque 10).
    - Hard-clip por ABS_MAX_UV. Se descartan epochs donde |x|max>ABS_MAX_UV.

5. Resumen:

    - Totales antes/después, % rechazo y top canales.

El orden de prepro minimize riesgos: notch antes de CAR/ICA, ICA en banda amplia, luego band-pass para MI.

de-dup de eventos evita inflar conteos por marcas duplicadas.

Rechazo mixto (p2p + absoluto) es más robusto que uno solo.


In [ ]:
# Crear epochs (0.5–4.5 s) con T1/T2 desde anotaciones (sin inflar) + fallback .edf.event
def create_epochs(edf_path, tmin=TMIN, tmax=TMAX):
    """
    Crea epochs para un EDF con diagnóstico claro y SIN inflar el nº de trials:
    - Preprocesa: soft-clip, notch, CAR+interp, ICA (opcional), band-pass (8–30 Hz).
    - Construye eventos T1/T2 manualmente DESDE raw.annotations (normalizando etiquetas),
      evitando duplicados cercanos en el tiempo; si falta cualquiera → fallback a .edf.event.
    - Mapea por RUN: LR → {T1=Left(1), T2=Right(2)}, OF → {T1=Both Fists(3), T2=Both Feet(4)}.
    - Epoching (0.5–4.5 s), rechazo por pico-a-pico y hard-clip robusto.
    - Devuelve (epochs, resumen) y prints de diagnóstico (fuente usada, conteos, etc.).
    """
    run = parse_subject_run(edf_path)
    kind = run_kind(run)  # 'LR' | 'OF'
    if kind is None:
        raise RuntimeError(f'Run {run} no es MI (LR/OF): {edf_path}')

    # ----------------- 1) Carga + prepro -----------------
    raw = load_raw(edf_path)
    soft_clip_inplace(raw, limit_uv=SOFTCLIP_DEFAULT); ensure_finite_raw(raw, where='tras carga')
    raw = apply_notch(raw);                   ensure_finite_raw(raw, where='tras notch')
    raw = apply_car_and_interpolation(raw);   ensure_finite_raw(raw, where='tras CAR/interpolacion')
    raw = apply_ica(raw);                     ensure_finite_raw(raw, where='tras ICA')
    soft_clip_inplace(raw, limit_uv=SOFTCLIP_DEFAULT)
    raw = apply_bandpass(raw);                ensure_finite_raw(raw, where='tras band-pass')
    if isinstance(RESAMPLE_TO, (int, float)) and RESAMPLE_TO > 0:
        raw.resample(RESAMPLE_TO, npad="auto")

    # ----------------- 2) Eventos T1/T2 -----------------
    # Normalizador de etiquetas (e.g., "t 1" -> "T1")
    def _norm_tag(s: str) -> str:
        return str(s).strip().upper().replace(' ', '')

    # a) Diagnóstico de anotaciones crudas
    if raw.annotations is not None and len(raw.annotations) > 0:
        from collections import Counter
        desc_norm = [_norm_tag(d) for d in raw.annotations.description]
        cnt_anno = Counter(desc_norm)
        print(f"  ↳ Anotaciones EDF (raw): {dict(cnt_anno)}")
    else:
        print("  ↳ Anotaciones EDF (raw): <sin anotaciones>")

    # b) Intento principal: construir eventos manualmente desde raw.annotations
    #    (evita la duplicación que a veces aparece con events_from_annotations)
    sf = float(raw.info['sfreq'])
    onsets_sec = list(raw.annotations.onset) if raw.annotations is not None else []
    desc_norm  = [_norm_tag(d) for d in (raw.annotations.description if raw.annotations is not None else [])]

    # Recolectar T1 y T2 (en segundos) y filtrar duplicados demasiado cercanos
    T1, T2 = [], []
    for onset, tag in zip(onsets_sec, desc_norm):
        if tag == 'T1':
            T1.append(float(onset))
        elif tag == 'T2':
            T2.append(float(onset))
    # Orden y anti-duplicados (p. ej. si hay marcas dobles a <0.5s)
    def _dedup(sorted_list, min_gap_sec=0.5):
        keep = []
        last = None
        for x in sorted(sorted_list):
            if last is None or (x - last) >= min_gap_sec:
                keep.append(x)
                last = x
        return keep
    T1 = _dedup(T1, min_gap_sec=0.5)
    T2 = _dedup(T2, min_gap_sec=0.5)

    used_source = "annotations"
    events = None
    mapping = None

    # ¿Hay ambos códigos presentes vía anotaciones?
    if len(T1) > 0 and len(T2) > 0:
        ev_list = []
        for t in T1:
            # LR: T1→1 (Left) | OF: T1→3 (Both Fists)
            code = 1 if kind == 'LR' else 3
            ev_list.append([int(round((raw.first_time + t) * sf)), 0, code])
        for t in T2:
            # LR: T2→2 (Right) | OF: T2→4 (Both Feet)
            code = 2 if kind == 'LR' else 4
            ev_list.append([int(round((raw.first_time + t) * sf)), 0, code])
        ev_list.sort(key=lambda r: r[0])
        events = np.asarray(ev_list, dtype=int)
        mapping = {'Left': 1, 'Right': 2} if kind == 'LR' else {'Both Fists': 3, 'Both Feet': 4}
    else:
        # c) Fallback: leer .edf.event y reconstruir (si falta cualquiera)
        rows = read_event_file_fallback(Path(edf_path))
        if rows:
            ev_list = []
            t1_rows = t2_rows = 0
            for onset_sec, code in rows:
                cc = _norm_tag(code)
                if cc == 'T1':
                    t1_rows += 1
                    code_num = 1 if kind == 'LR' else 3
                    ev_list.append([int(round((raw.first_time + onset_sec) * sf)), 0, code_num])
                elif cc == 'T2':
                    t2_rows += 1
                    code_num = 2 if kind == 'LR' else 4
                    ev_list.append([int(round((raw.first_time + onset_sec) * sf)), 0, code_num])
            if ev_list:
                # anti-duplicados también aquí por si el .event tiene marcas pegadas
                ev_list.sort(key=lambda r: r[0])
                # compactar por código si hay duplicados a <0.5 s
                compact = []
                last_by_code = {}
                for s, _, c in ev_list:
                    last_s = last_by_code.get(c, -1e9)
                    if (s - last_s) >= int(0.5 * sf):  # 0.5 s en muestras
                        compact.append([s, 0, c])
                        last_by_code[c] = s
                events = np.asarray(compact, dtype=int)
                mapping = {'Left': 1, 'Right': 2} if kind == 'LR' else {'Both Fists': 3, 'Both Feet': 4}
                used_source = ".edf.event"
            else:
                raise RuntimeError(f'Fallback .edf.event no halló T1/T2 en {Path(edf_path).name}')
        else:
            raise RuntimeError(f'Sin T1/T2 (ni en anotaciones ni en .edf.event) en {Path(edf_path).name}')

    # Log breve de conteos por clase ANTES del rechazo de artefactos
    try:
        # Contar por código en el array events
        cnt_left  = int(np.sum(events[:, 2] == (1 if kind == 'LR' else 3)))
        cnt_right = int(np.sum(events[:, 2] == (2 if kind == 'LR' else 4)))
        cls_counts = ({"Left": cnt_left, "Right": cnt_right} if kind == 'LR'
                      else {"Both Fists": cnt_left, "Both Feet": cnt_right})
    except Exception:
        cls_counts = {}
    print(f"  ↳ Fuente usada: {used_source} | Épocas por clase: {cls_counts} | Total={sum(cls_counts.values())}")

    # ----------------- 3) Epoching -----------------
    # Evitamos descartar por anotaciones genéricas (para no perder trials válidos)
    reject_by_ann = False
    epochs = mne.Epochs(
        raw, events, event_id=mapping,
        tmin=tmin, tmax=tmax,
        baseline=BASELINE,
        picks='eeg',
        preload=True,
        verbose=False,
        reject_by_annotation=reject_by_ann
    )
    n_total = len(epochs)

    # ----------------- 4) Rechazo artefactos -----------------
    # Pico-a-pico por canal
    reject = dict(eeg=PEAK_TO_PEAK_UV * 1e-6)
    epochs_tmp = epochs.copy()
    epochs_tmp.drop_bad(reject=reject)
    n_after_dropbad = len(epochs_tmp)

    # Métrica por canales usando drop_log
    top_channels = summarize_drop_log(
        epochs_tmp.drop_log, n_total, epochs_tmp.info['ch_names']
    )

    # Hard-clip por amplitud absoluta (si se activa)
    epochs = epochs_tmp
    if ABS_MAX_UV is not None and len(epochs) > 0:
        X = epochs.get_data() * 1e6  # uV
        keep_idx = [i for i in range(len(epochs)) if np.max(np.abs(X[i])) <= ABS_MAX_UV]
        if len(keep_idx) < len(epochs):
            epochs = epochs[keep_idx]


    n_after_hardclip  = len(epochs)
    n_rejected_total  = n_total - n_after_hardclip
    pct_rejected_total = (100.0 * n_rejected_total / n_total) if n_total > 0 else 0.0

    resumen = {
        'run': run,
        'n_total': n_total,
        'n_after_dropbad': n_after_dropbad,
        'n_after_hardclip': n_after_hardclip,
        'n_rejected_total': n_rejected_total,
        'pct_rejected_total': pct_rejected_total,
        'top_channels': top_channels,  # lista de (canal, conteo, %)
    }
    return epochs, resumen


## Procesamiento por sujeto, guardado y resumen

Para cada sujeto valido, buscamos sus runs MI (LR y OF), generamos epochs, concatenamos, guardamos y registramos un resumen por clase.


In [ ]:
# Procesar sujetos: guardar epochs y resumen subjectssummary
#  — verificación de IDs fijos 1..4 y auditoría antes de guardar

TARGET_EVENT_MAP = {'Left': 1, 'Right': 2, 'Both Fists': 3, 'Both Feet': 4}

def verify_event_ids_fixed(epochs, target_map=TARGET_EVENT_MAP, verbose=True):
    """
    Verifica que:
      - epochs.event_id sea un subconjunto de target_map (IDs fijos 1..4)
      - epochs.events[:,2] contenga solo IDs presentes en epochs.event_id
    No modifica los datos. Lanza ValueError si detecta inconsistencias.
    Devuelve un dict con conteos por clase (para log/auditoría).
    """
    eid = epochs.event_id.copy()
    # 1) todas las clases conocidas y sus códigos coinciden con target_map
    for cls, code in eid.items():
        if cls not in target_map:
            raise ValueError(f"Clase inesperada en event_id: {cls}")
        if target_map[cls] != code:
            raise ValueError(f"Código inesperado para {cls}: {code} (esperado {target_map[cls]})")

    # 2) los códigos en events son solo los del event_id
    valid_codes = set(eid.values())
    codes_in_events = set(np.unique(epochs.events[:, 2]))
    if not codes_in_events.issubset(valid_codes):
        raise ValueError(f"Códigos en events {sorted(codes_in_events)} no son subconjunto de {sorted(valid_codes)}")

    # 3) conteos por clase
    counts = {cls: len(epochs[cls]) for cls in eid.keys()}
    if verbose:
        print("  ✔ Verificación IDs 1..4 OK")
        print("  event_id:", eid)
        print("  Conteos por clase:", counts)
    return counts

subjects = list_subjects()
summary_rows = []

for s in subjects:
    print(f"\n▶ Sujeto S{s:03d}")
    edf_lr = [DATA_RAW / f'S{s:03d}' / f'S{s:03d}R{r:02d}.edf' for r in MI_RUNS_LR
              if (DATA_RAW / f'S{s:03d}' / f'S{s:03d}R{r:02d}.edf').exists()]
    edf_of = [DATA_RAW / f'S{s:03d}' / f'S{s:03d}R{r:02d}.edf' for r in MI_RUNS_OF
              if (DATA_RAW / f'S{s:03d}' / f'S{s:03d}R{r:02d}.edf').exists()]
    if not edf_lr or not edf_of:
        print(" Faltan runs MI (LR u OF). Saltando.")
        continue

    epochs_list = []
    run_summaries = []  # para el TXT por run
    template_ch_names = None  # para fijar orden de canales dentro del sujeto

    for p in edf_lr + edf_of:
        try:
            ep, res = create_epochs(p)

            # Fijar orden de canales con base en el primer run válido del sujeto
            if template_ch_names is None:
                template_ch_names = ep.ch_names.copy()
            elif ep.ch_names != template_ch_names:
                ep = ep.copy().reorder_channels(template_ch_names)

            # Resample opcional a nivel epochs (si no lo hiciste en raw)
            if isinstance(RESAMPLE_TO, (int, float)) and RESAMPLE_TO > 0:
                if not np.isclose(ep.info['sfreq'], RESAMPLE_TO, rtol=0, atol=1e-6):
                    ep = ep.copy().resample(RESAMPLE_TO, npad="auto")

            # Reporte por consola
            print(f"  {p.name}: {len(ep)} epochs (total {res['n_total']}, rechazados {res['n_rejected_total']} = {res['pct_rejected_total']:.1f}%)")

            if len(ep) > 0:
                epochs_list.append(ep)
            run_summaries.append((p.name, res))

        except Exception as e:
            print("  Error", p.name, ":", e)

    # Si no quedó nada válido, saltar sujeto
    if not epochs_list:
        print(" Sin epochs válidos en ningún run. Saltando sujeto.")
        continue

    # Concatenar epochs del sujeto
    epochs_subj = mne.concatenate_epochs(epochs_list, on_mismatch='ignore')

    # === Verificación de IDs fijos 1..4 antes de guardar (no modifica) ===
    counts_after = verify_event_ids_fixed(epochs_subj, verbose=True)

    # Guardar auditoría mínima (IDs y conteos por clase)
    audit_rows = []
    for cls in ['Left','Right','Both Fists','Both Feet']:
        if cls in epochs_subj.event_id:
            audit_rows.append({
                'subject': f"S{s:03d}",
                'class': cls,
                'code': epochs_subj.event_id[cls],
                'count': counts_after.get(cls, 0),
            })
    audit_df = pd.DataFrame(audit_rows)
    audit_csv = SUBJ_SUMMARY_DIR / f"S{s:03d}_eventid_audit.csv"
    audit_df.to_csv(audit_csv, index=False)
    print("  Auditoría de IDs guardada en:", audit_csv)

    # Guardar epochs del sujeto
    out_path = DATA_PROC / f'S{s:03d}_MI-epo.fif'
    epochs_subj.save(out_path, overwrite=True)
    print(' Guardado:', out_path)

    # ---- Resumen por sujeto (TXT con detalle por run) ----
    lines = []
    for fname, res in sorted(run_summaries, key=lambda x: x[1]['run']):
        total = res['n_total']
        rej = res['n_rejected_total']
        pct = res['pct_rejected_total']
        top_str = format_top_channels(res['top_channels'])
        lines.append(f"{fname}")
        lines.append(f"epochs totales: {total}")
        lines.append(f"rechazados: {rej} ({pct:.1f}%)")
        lines.append(f"principales canales: {top_str}")
        lines.append("")  # línea en blanco
    # verificación rápida al final del TXT
    lines.append("event_id (final): " + str(epochs_subj.event_id))
    lines.append("conteos por clase (final): " + str(counts_after))

    txt_path = SUBJ_SUMMARY_DIR / f"S{s:03d}_summary.txt"
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(lines))
    print(" Resumen por run guardado en:", txt_path)

    # ---- Conteo por clase (CSV) ----
    class_counts = []
    for k in ['Left','Right','Both Fists','Both Feet']:
        if k in epochs_subj.event_id:
            class_counts.append({'y_class': k, 'count': len(epochs_subj[k])})
    df_cls = pd.DataFrame(class_counts)
    csv_out = SUBJ_SUMMARY_DIR / f"S{s:03d}_class_counts.csv"
    df_cls.to_csv(csv_out, index=False)
    print(" Conteo por clase guardado en:", csv_out)

    # ---- (opcional) agregar fila a un resumen global en memoria ----
    row = {'subject': s, 'n_epochs': len(epochs_subj)}
    for d in class_counts:
        row[d['y_class']] = d['count']
    summary_rows.append(row)

# Resumen global (opcional)
if summary_rows:
    df_sum_all = pd.DataFrame(summary_rows).set_index('subject').sort_index()
    display(df_sum_all)
    df_sum_all.to_csv(REPORT_TAB / 'preproc_epochs_count_per_subject.csv')


Sujetos validos: 103

▶ Sujeto S001
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T2': 7, 'T1': 8}
  ↳ Fuente usada: annotations | Épocas por clase: {'Left': 8, 'Right': 7} | Total=15
  S001R04.edf: 15 epochs (total 15, rechazados 0 = 0.0%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T1': 8, 'T2': 7}
  ↳ Fuente usada: annotations | Épocas por clase: {'Left': 8, 'Right': 7} | Total=15
  S001R08.edf: 14 epochs (total 15, rechazados 1 = 6.7%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T2': 8, 'T1': 7}
  ↳ Fuente usada: annotations | Épocas por clase: {'Left': 7, 'Right': 8} | Total=15
  S001R12.edf: 15 epochs (total 15, rechazados 0 = 0.0%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T2': 8, 'T1': 7}
  ↳ Fuente usada: annotations | Épocas por clase: {'Both Fists': 7, 'Both Feet': 8} | Total=15
  S001R06.edf: 15 epochs (total 15, rechazados 0 = 0.0%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T1': 7, 'T2': 8}
  ↳ Fuente usada: annotations | Épocas por clase: {'Both Fists': 7, 'Both Feet': 8} | Total=15
  S001R10.e

C:\Users\joelc\AppData\Local\Temp\ipykernel_32400\3843191980.py:84: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_subj = mne.concatenate_epochs(epochs_list, on_mismatch='ignore')


  ↳ Anotaciones EDF (raw): {'T0': 15, 'T1': 7, 'T2': 8}
  ↳ Fuente usada: annotations | Épocas por clase: {'Left': 7, 'Right': 8} | Total=15
  S002R04.edf: 14 epochs (total 14, rechazados 0 = 0.0%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T1': 8, 'T2': 7}
  ↳ Fuente usada: annotations | Épocas por clase: {'Left': 8, 'Right': 7} | Total=15
  S002R08.edf: 14 epochs (total 14, rechazados 0 = 0.0%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T1': 8, 'T2': 7}
  ↳ Fuente usada: annotations | Épocas por clase: {'Left': 8, 'Right': 7} | Total=15
  S002R12.edf: 14 epochs (total 14, rechazados 0 = 0.0%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T1': 8, 'T2': 7}
  ↳ Fuente usada: annotations | Épocas por clase: {'Both Fists': 8, 'Both Feet': 7} | Total=15
  S002R06.edf: 14 epochs (total 14, rechazados 0 = 0.0%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T2': 7, 'T1': 8}
  ↳ Fuente usada: annotations | Épocas por clase: {'Both Fists': 8, 'Both Feet': 7} | Total=15
  S002R10.edf: 14 epochs (total 14, rechazados 

C:\Users\joelc\AppData\Local\Temp\ipykernel_32400\3843191980.py:84: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_subj = mne.concatenate_epochs(epochs_list, on_mismatch='ignore')


  ↳ Anotaciones EDF (raw): {'T0': 15, 'T2': 7, 'T1': 8}
  ↳ Fuente usada: annotations | Épocas por clase: {'Left': 8, 'Right': 7} | Total=15
  S003R04.edf: 15 epochs (total 15, rechazados 0 = 0.0%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T1': 7, 'T2': 8}
  ↳ Fuente usada: annotations | Épocas por clase: {'Left': 7, 'Right': 8} | Total=15
  S003R08.edf: 13 epochs (total 15, rechazados 2 = 13.3%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T2': 7, 'T1': 8}
  ↳ Fuente usada: annotations | Épocas por clase: {'Left': 8, 'Right': 7} | Total=15
  S003R12.edf: 15 epochs (total 15, rechazados 0 = 0.0%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T2': 8, 'T1': 7}
  ↳ Fuente usada: annotations | Épocas por clase: {'Both Fists': 7, 'Both Feet': 8} | Total=15
  S003R06.edf: 11 epochs (total 15, rechazados 4 = 26.7%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T1': 7, 'T2': 8}
  ↳ Fuente usada: annotations | Épocas por clase: {'Both Fists': 7, 'Both Feet': 8} | Total=15
  S003R10.edf: 14 epochs (total 15, rechazado

...

  ↳ Anotaciones EDF (raw): {'T0': 15, 'T2': 7, 'T1': 8}
  ↳ Fuente usada: annotations | Épocas por clase: {'Both Fists': 8, 'Both Feet': 7} | Total=15
  S109R06.edf: 0 epochs (total 14, rechazados 14 = 100.0%)


C:\Users\joelc\AppData\Local\Temp\ipykernel_32400\1456214442.py:149: RuntimeWarning: All epochs were dropped!
You might need to alter reject/flat-criteria or drop bad channels to avoid this. You can use Epochs.plot_drop_log() to see which channels are responsible for the dropping of epochs.
  epochs_tmp.drop_bad(reject=reject)


  ↳ Anotaciones EDF (raw): {'T0': 15, 'T1': 8, 'T2': 7}
  ↳ Fuente usada: annotations | Épocas por clase: {'Both Fists': 8, 'Both Feet': 7} | Total=15
  S109R10.edf: 14 epochs (total 14, rechazados 0 = 0.0%)
  ↳ Anotaciones EDF (raw): {'T0': 15, 'T1': 7, 'T2': 8}
  ↳ Fuente usada: annotations | Épocas por clase: {'Both Fists': 7, 'Both Feet': 8} | Total=15
  S109R14.edf: 0 epochs (total 14, rechazados 14 = 100.0%)
  ✔ Verificación IDs 1..4 OK
  event_id: {'Left': 1, 'Right': 2, 'Both Fists': 3, 'Both Feet': 4}
  Conteos por clase: {'Left': 4, 'Right': 5, 'Both Fists': 7, 'Both Feet': 7}
  Auditoría de IDs guardada en: C:\Users\joelc\Desktop\eeg2\data\processed\subjectssummary\S109_eventid_audit.csv
 Guardado: C:\Users\joelc\Desktop\eeg2\data\processed\S109_MI-epo.fif
 Resumen por run guardado en: C:\Users\joelc\Desktop\eeg2\data\processed\subjectssummary\S109_summary.txt
 Conteo por clase guardado en: C:\Users\joelc\Desktop\eeg2\data\processed\subjectssummary\S109_class_counts.csv


C:\Users\joelc\AppData\Local\Temp\ipykernel_32400\1456214442.py:149: RuntimeWarning: All epochs were dropped!
You might need to alter reject/flat-criteria or drop bad channels to avoid this. You can use Epochs.plot_drop_log() to see which channels are responsible for the dropping of epochs.
  epochs_tmp.drop_bad(reject=reject)
C:\Users\joelc\AppData\Local\Temp\ipykernel_32400\3843191980.py:84: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_subj = mne.concatenate_epochs(epochs_list, on_mismatch='ignore')


,n_epochs,Left,Right,Both Fists,Both Feet
subject,,,,,
1,88,23,21,21,23
2,84,21,21,21,21
3,83,23,20,20,20
4,72,16,17,20,19
5,84,21,21,21,21
...,...,...,...,...,...
105,84,21,21,21,21
106,90,24,21,23,22
107,73,20,14,17,22


## Revision de ICA



In [ ]:
# KPIs de limpieza y señal por sujeto (y resumen global)

import numpy as np
import pandas as pd
import mne
from pathlib import Path
from mne.time_frequency import psd_array_welch

# --- Config de ventanas y bandas ---
# Epochs están en [0.5, 4.5] s. Comparamos "temprano vs tardío" dentro de la tarea.
EARLY_WIN = (0.5, 1.5)   # ventana "temprana" dentro del ensayo
LATE_WIN  = (2.5, 3.5)   # ventana "tardía"   dentro del ensayo

MU_BAND    = (8., 12.)
BETA_BAND  = (13., 30.)
EMG_BAND   = (20., 40.)

# Canales frontales usados como proxy de EOG (si existen)
EOG_PROXIES = ("Fp2", "Fp1", "AF7", "AF8")

# --- Rutas de entrada/salida ---
EPO_GLOB     = str(DATA_PROC / 'S???_MI-epo.fif')  # generados en bloque 13
OUT_PER_SUBJ = REPORT_TAB / 'kpi_clean_signal_per_subject_peak300.csv'
OUT_GLOBAL   = REPORT_TAB / 'kpi_clean_signal_GLOBAL_peak300.csv'
REPORT_TAB.mkdir(parents=True, exist_ok=True)

def _next_pow2(n: int) -> int:
    """Siguiente potencia de 2 >= n."""
    p = 1
    while p < n:
        p <<= 1
    return p

def _welch_psd(X, sfreq, fmin, fmax, prefer_seg=256):
    """
    X: ndarray (N, T) en VOLTS.
    Ajusta n_per_seg, n_fft y n_overlap según T para evitar errores.
    """
    T = X.shape[-1]
    # n_per_seg no puede exceder T
    n_per_seg = min(prefer_seg, T)
    # Para ventanas muy cortas, baja el tamaño
    if n_per_seg < 32:  # evita segmentos demasiado pequeños
        n_per_seg = max(16, T)  # como mínimo usa todo T
    # n_fft puede ser >= n_per_seg; usamos potencia de 2 para eficiencia
    n_fft = _next_pow2(n_per_seg)
    # solape típico 50%
    n_overlap = n_per_seg // 2

    psd, freqs = psd_array_welch(
        X, sfreq=sfreq, fmin=fmin, fmax=fmax,
        n_per_seg=n_per_seg, n_fft=n_fft, n_overlap=n_overlap,
        average='mean', verbose=False
    )
    return psd, freqs

def _pick_frontal_existing(ch_names):
    """Devuelve el primer canal frontal disponible como proxy de EOG, o None."""
    for c in EOG_PROXIES:
        if c in ch_names:
            return c
    return None

def _time_mask(n_times, sfreq, tmin, tmax):
    """Máscara temporal en índices para ventana [tmin, tmax) sobre la escala de muestras."""
    t = np.arange(n_times) / sfreq
    return (t >= tmin) & (t < tmax)


def _bandpower_ratio(data_v, sfreq, band=(20,40), fmin=1., fmax=45., prefer_seg=256):
    """
    data_v: (n_epochs, n_channels, n_times) en VOLTS.
    Devuelve fracción media de potencia en 'band' respecto a potencia total [fmin, fmax].
    """
    X = data_v.reshape(data_v.shape[0]*data_v.shape[1], -1)  # (N, T)
    psd, freqs = _welch_psd(X, sfreq, fmin=fmin, fmax=fmax, prefer_seg=prefer_seg)
    fmask_band  = (freqs >= band[0]) & (freqs <= band[1])
    bp_band  = np.trapezoid(psd[:, fmask_band], freqs[fmask_band], axis=1)
    bp_total = np.trapezoid(psd, freqs, axis=1)
    ratio = bp_band / (bp_total + 1e-12)
    return float(np.mean(ratio))


def _erd_percent_between_windows(data_v, sfreq, band, early_win=EARLY_WIN, late_win=LATE_WIN):
    """
    ERD% = 100*(P_late - P_early)/(P_early + eps)
    Usa Welch seguro por ventana (sub-segmentos más cortos que el epoch completo).
    """
    n_ep, n_ch, n_t = data_v.shape
    w_early = _time_mask(n_t, sfreq, early_win[0], early_win[1])
    w_late  = _time_mask(n_t, sfreq, late_win[0],  late_win[1])

    def _bandpower(seg):
        # seg: (n_ep, n_ch, T_win) en V
        X = seg.reshape(n_ep*n_ch, -1)
        # en ventanas cortas mejor un prefer_seg menor (p.ej. 128)
        psd, freqs = _welch_psd(X, sfreq, fmin=band[0], fmax=band[1], prefer_seg=128)
        return np.mean(np.trapezoid(psd, freqs, axis=1))

    p_early = _bandpower(data_v[:, :, w_early]) if w_early.any() else 0.0
    p_late  = _bandpower(data_v[:, :, w_late])  if w_late.any()  else 0.0
    erd = 100.0 * (p_late - p_early) / (p_early + 1e-12)
    return float(erd)

def _eog_proxy_p2p_uv(epochs):
    """
    Estima residuo EOG con pico-a-pico (µV) en un canal frontal proxy si existe.
    Si no hay frontal, devuelve NaN.
    """
    ch = _pick_frontal_existing(epochs.ch_names)
    if ch is None:
        return np.nan
    X = epochs.get_data(picks=[ch]) * 1e6  # µV
    # pico-a-pico máximo entre todos los epochs
    p2p = np.max(X) - np.min(X)
    return float(p2p)

def compute_kpis_for_epochs(ep):
    """
    Calcula KPIs principales para un objeto mne.Epochs:
      - sfreq, n_epochs, n_channels
      - eog_proxy_p2p_uv
      - emg_ratio (20-40 Hz)
      - erd_mu_pct, erd_beta_pct (ventanas early vs late)
    """
    sf = float(ep.info['sfreq'])
    X = ep.get_data()  # VOLTS (n_epochs, n_channels, n_times)

    kpis = {
        'sfreq': sf,
        'n_epochs': int(len(ep)),
        'n_channels': int(len(ep.ch_names)),
        'eog_p2p_uv': _eog_proxy_p2p_uv(ep),
        'emg_ratio_20_40': _bandpower_ratio(X, sf, band=EMG_BAND),
        'erd_mu_pct': _erd_percent_between_windows(X, sf, band=MU_BAND),
        'erd_beta_pct': _erd_percent_between_windows(X, sf, band=BETA_BAND),
    }
    return kpis

# --- Recorrido por sujetos ---
rows = []
epo_files = sorted(Path(DATA_PROC).glob('S???_MI-epo.fif'))
print(f"Archivos de epochs por sujeto encontrados: {len(epo_files)}")

for f in epo_files:
    sid = int(f.stem[1:4])  # S001_MI-epo.fif -> 1
    try:
        ep = mne.read_epochs(f, preload=True, verbose=False)
        k = compute_kpis_for_epochs(ep)
        k['subject'] = sid
        rows.append(k)
        print(f"S{sid:03d} -> n_ep={k['n_epochs']} | EOG p2p={k['eog_p2p_uv']:.1f} µV | EMG ratio={k['emg_ratio_20_40']:.3f} | ERD mu={k['erd_mu_pct']:.1f}% | ERD beta={k['erd_beta_pct']:.1f}%")
    except Exception as e:
        print(f"No pude procesar {f.name}: {e}")

# --- Salidas ---
if rows:
    df_kpi = pd.DataFrame(rows).sort_values('subject').reset_index(drop=True)
    display(df_kpi.head(10))
    df_kpi.to_csv(OUT_PER_SUBJ, index=False)
    print("KPIs por sujeto guardados en:", OUT_PER_SUBJ)

    # Resumen global (medias y DE)
    stats = df_kpi.drop(columns=['subject']).agg(['mean','std']).T
    stats = stats.rename(columns={'mean':'mean', 'std':'std'})
    display(stats)
    stats.to_csv(OUT_GLOBAL)
    print("KPIs globales guardados en:", OUT_GLOBAL)
else:
    print("No se generaron KPIs; ¿existen S###_MI-epo.fif en", DATA_PROC, "?")


Archivos de epochs por sujeto encontrados: 103
S001 -> n_ep=88 | EOG p2p=169.3 µV | EMG ratio=0.256 | ERD mu=-11.9% | ERD beta=-5.4%
S002 -> n_ep=84 | EOG p2p=92.8 µV | EMG ratio=0.328 | ERD mu=23.7% | ERD beta=11.4%
S003 -> n_ep=83 | EOG p2p=309.2 µV | EMG ratio=0.277 | ERD mu=-21.9% | ERD beta=-1.6%
S004 -> n_ep=72 | EOG p2p=47.6 µV | EMG ratio=0.384 | ERD mu=9.5% | ERD beta=5.2%
S005 -> n_ep=84 | EOG p2p=118.1 µV | EMG ratio=0.380 | ERD mu=-3.1% | ERD beta=2.6%
S006 -> n_ep=82 | EOG p2p=97.8 µV | EMG ratio=0.584 | ERD mu=-1.8% | ERD beta=6.8%
S007 -> n_ep=89 | EOG p2p=118.9 µV | EMG ratio=0.121 | ERD mu=91.4% | ERD beta=7.7%
S008 -> n_ep=84 | EOG p2p=105.5 µV | EMG ratio=0.400 | ERD mu=0.9% | ERD beta=2.4%
S009 -> n_ep=2 | EOG p2p=207.4 µV | EMG ratio=0.404 | ERD mu=-26.9% | ERD beta=-5.8%
S010 -> n_ep=83 | EOG p2p=187.9 µV | EMG ratio=0.231 | ERD mu=-0.4% | ERD beta=9.3%
S011 -> n_ep=83 | EOG p2p=35.9 µV | EMG ratio=0.261 | ERD mu=34.9% | ERD beta=4.3%
S012 -> n_ep=84 | EOG p2p=174

,sfreq,n_epochs,n_channels,eog_p2p_uv,emg_ratio_20_40,erd_mu_pct,erd_beta_pct,subject
0,160.0,88,64,169.301638,0.255985,-11.850425,-5.395321,1
1,160.0,84,64,92.752878,0.328500,23.662951,11.387435,2
2,160.0,83,64,309.168274,0.276510,-21.907394,-1.609647,3
3,160.0,72,64,47.568506,0.383672,9.548712,5.200087,4
4,160.0,84,64,118.061624,0.379763,-3.086678,2.642420,5
5,160.0,82,64,97.778840,0.583933,-1.782225,6.765377,6
6,160.0,89,64,118.887754,0.120505,91.392846,7.689940,7
7,160.0,84,64,105.512401,0.399698,0.938034,2.399878,8
8,160.0,2,64,207.365440,0.403719,-26.889564,-5.775373,9
9,160.0,83,64,187.946505,0.231196,-0.390699,9.292493,10


KPIs por sujeto guardados en: C:\Users\joelc\Desktop\eeg2\reports\tables\02_prepro\kpi_clean_signal_per_subject_peak300.csv


,mean,std
sfreq,160.000000,0.000000
n_epochs,74.718447,18.512009
n_channels,64.000000,0.000000
eog_p2p_uv,144.818866,82.512409
emg_ratio_20_40,0.287051,0.109398
erd_mu_pct,15.457889,41.645950
erd_beta_pct,1.031325,7.473197


KPIs globales guardados en: C:\Users\joelc\Desktop\eeg2\reports\tables\02_prepro\kpi_clean_signal_GLOBAL_peak300.csv


## Resumen global de clases

Itera sujetos válidos y reúne sus 6 runs MI (3 LR + 3 OF).

Llama create_epochs por run y homogeneiza orden de canales dentro del sujeto.

(Opcional) resample a nivel epochs si no se hizo en raw.

Concatena epochs del sujeto.

Verifica que los IDs de clases coinciden con el mapa fijo {Left:1, Right:2, Both Fists:3, Both Feet:4}.

Guarda:

    - S###_MI-epo.fif (epochs concatenados por sujeto).
    - S###_eventid_audit.csv (conteos por clase).
    - S###_summary.txt (resumen por run: totales, rechazados, canales top).
    - S###_class_counts.csv (conteo final por clase).
    - Crea un resumen global en memoria y CSV.

In [ ]:
# Resumen global de clases (lee subjectssummary/*_class_counts.csv)
from pathlib import Path
import pandas as pd

# Carpeta donde ya estás guardando los CSV por sujeto
SSUM = SUBJ_SUMMARY_DIR

rows = []
for csv_path in sorted(SSUM.glob("S???_class_counts.csv")):
    sid = int(csv_path.stem.split("_")[0][1:])  # S001 -> 1
    df = pd.read_csv(csv_path)
    # Normalizamos nombres por si acaso
    df['y_class'] = df['y_class'].str.strip()
    # Pivot por clase -> conteos
    wide = df.pivot_table(index=None, columns='y_class', values='count', aggfunc='sum').fillna(0).astype(int)
    rec = {'subject': sid}
    for c in ['Left','Right','Both Fists','Both Feet']:
        rec[c] = int(wide.get(c, pd.Series([0])).iloc[0]) if c in wide else 0
    rec['Total'] = rec['Left'] + rec['Right'] + rec['Both Fists'] + rec['Both Feet']
    rows.append(rec)

if rows:
    df_global = pd.DataFrame(rows).set_index('subject').sort_index()
    # Guarda tabla por sujeto
    out_per_subject = REPORT_TAB / 'preproc_class_counts_per_subject.csv'
    df_global.to_csv(out_per_subject)
    display(df_global.head(10))
    print("Tabla por sujeto guardada en:", out_per_subject)

    # Totales globales por clase
    totals = df_global[['Left','Right','Both Fists','Both Feet']].sum().rename('count').to_frame()
    out_totals = REPORT_TAB / 'preproc_class_counts_GLOBAL.csv'
    totals.to_csv(out_totals)
    display(totals.T)
    print("Totales globales guardados en:", out_totals)

    # (Opcional) ranking de sujetos por total de epochs
    top_more = df_global.sort_values('Total', ascending=False).head(5)
    top_less = df_global.sort_values('Total', ascending=True).head(5)
    print("\nTop 5 sujetos con más epochs:\n", top_more)
    print("\nTop 5 sujetos con menos epochs:\n", top_less)

    # (Opcional) marca sujetos candidatos a exclusión (criterio simple)
    MIN_PER_CLASS = 10   # ajusta a tu criterio (p. ej., 8–12)
    mask_bad = (df_global[['Left','Right','Both Fists','Both Feet']] < MIN_PER_CLASS).any(axis=1)
    bad_subjects = df_global.index[mask_bad].tolist()
    out_bad = REPORT_TAB / 'preproc_subjects_flagged_low_counts.txt'
    with open(out_bad, 'w') as f:
        for sid in bad_subjects:
            f.write(f"S{sid:03d}\n")
    print(f"\nSujetos marcados por <{MIN_PER_CLASS} ensayos en alguna clase:", bad_subjects)
    print("Lista guardada en:", out_bad)
else:
    print("No se encontraron *_class_counts.csv en", SSUM)


,Left,Right,Both Fists,Both Feet,Total
subject,,,,,
1,23,21,21,23,88
2,21,21,21,21,84
3,23,20,20,20,83
4,16,17,20,19,72
5,21,21,21,21,84
6,20,20,21,21,82
7,23,22,21,23,89
8,21,21,21,21,84
9,1,0,1,0,2


Tabla por sujeto guardada en: C:\Users\joelc\Desktop\eeg2\reports\tables\02_prepro\preproc_class_counts_per_subject.csv


,Left,Right,Both Fists,Both Feet
count,1939,1915,1900,1942


Totales globales guardados en: C:\Users\joelc\Desktop\eeg2\reports\tables\02_prepro\preproc_class_counts_GLOBAL.csv

Top 5 sujetos con más epochs:
          Left  Right  Both Fists  Both Feet  Total
subject                                           
30         24     21          24         21     90
35         22     23          24         21     90
29         23     22          22         23     90
101        22     23          24         21     90
65         24     21          22         23     90

Top 5 sujetos con menos epochs:
          Left  Right  Both Fists  Both Feet  Total
subject                                           
9           1      0           1          0      2
13          5      3           5          5     18
109         4      5           7          7     23
24          7      5           8          4     24
48          4      9           8          8     29

Sujetos marcados por <10 ensayos en alguna clase: [9, 13, 17, 19, 23, 24, 44, 48, 53, 60, 79, 99, 109]


## Evaluacion de sujetos

In [ ]:
# Resumen de retención por sujeto, listas Strict OK / DROP y totales globales por clase
import re, io
from pathlib import Path

# --- Parámetros de criterio "Strict" (ajusta a tu gusto) ---
STRICT_MIN_RETENTION = 50.0     # % mínimo de retención total por sujeto
STRICT_MIN_EPOCHS_PER_CLASS = 10  # mínimo de épocas retenidas por clase

# Carpetas de entrada/salida
summary_dir = SUBJ_SUMMARY_DIR                     # data/processed/subjectssummary
out_table   = REPORT_TAB / 'subjects_retention_summary.csv'
out_ok      = REPORT_TAB / 'subjects_strict_OK.txt'
out_drop    = REPORT_TAB / 'subjects_strict_DROP.txt'
out_global  = REPORT_TAB / 'global_class_totals.csv'
REPORT_TAB.mkdir(parents=True, exist_ok=True)

# Regex para parsear líneas del TXT por run:
#   epochs totales: 14
#   rechazados: 3 (21.4%)
_re_total   = re.compile(r'^epochs\s+totales:\s*(\d+)\s*$', re.I)
_re_rej     = re.compile(r'^rechazados:\s*(\d+)\s*\(', re.I)

def _read_subject_runs_totals(txt_path: Path):
    """
    Lee S###_summary.txt y acumula:
    - total_epochs_posibles = suma de "epochs totales" de sus 6 runs MI
    - total_rejected = suma de 'rechazados' reportados por run
    """
    if not txt_path.exists():
        return 0, 0
    total_possible = 0
    total_rejected = 0
    with open(txt_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.strip()
            m1 = _re_total.match(line)
            if m1:
                total_possible += int(m1.group(1))
                continue
            m2 = _re_rej.match(line)
            if m2:
                total_rejected += int(m2.group(1))
                continue
    return total_possible, total_rejected

def _read_subject_class_counts(csv_path: Path):
    """
    Lee S###_class_counts.csv -> dict por clase con conteo retenido.
    Devuelve con claves estándar (Left, Right, Both Fists, Both Feet).
    Si no existe, devuelve ceros.
    """
    wanted = ['Left','Right','Both Fists','Both Feet']
    counts = {k: 0 for k in wanted}
    if not csv_path.exists():
        return counts
    df = pd.read_csv(csv_path)
    # columnas esperadas: y_class, count
    for _, row in df.iterrows():
        cls = str(row.get('y_class', '')).strip()
        cnt = int(row.get('count', 0))
        if cls in counts:
            counts[cls] = cnt
    return counts

rows = []
strict_ok, strict_drop = [], []
global_counts = {'Left':0, 'Right':0, 'Both Fists':0, 'Both Feet':0}

# Recorre todos los sujetos que tengan TXT (o CSV) en subjectssummary
subjects_found = sorted({p.name[1:4] for p in summary_dir.glob('S???_summary.txt')} |
                        {p.name[1:4] for p in summary_dir.glob('S???_class_counts.csv')})

for sid in subjects_found:
    subj = int(sid)
    txt_path = summary_dir / f"S{sid}_summary.txt"
    csv_path = summary_dir / f"S{sid}_class_counts.csv"

    total_possible, total_rejected = _read_subject_runs_totals(txt_path)
    total_retained = max(0, total_possible - total_rejected)
    retention_pct = (100.0 * total_retained / total_possible) if total_possible > 0 else 0.0

    cls_counts = _read_subject_class_counts(csv_path)
    # acumular globales
    for k in global_counts:
        global_counts[k] += cls_counts.get(k, 0)

    # Decidir STRICT
    reasons = []
    if total_possible == 0:
        reasons.append("sin_runs")
    if retention_pct < STRICT_MIN_RETENTION:
        reasons.append(f"retencion<{STRICT_MIN_RETENTION:.0f}%")
    for k in ['Left','Right','Both Fists','Both Feet']:
        if cls_counts.get(k, 0) < STRICT_MIN_EPOCHS_PER_CLASS:
            reasons.append(f"{k}<{STRICT_MIN_EPOCHS_PER_CLASS}")

    strict_pass = (len(reasons) == 0)

    row = {
        'subject': subj,
        'epochs_total_possible': total_possible,
        'epochs_retained': total_retained,
        'retention_pct': round(retention_pct, 1),
        'Left': cls_counts['Left'],
        'Right': cls_counts['Right'],
        'Both Fists': cls_counts['Both Fists'],
        'Both Feet': cls_counts['Both Feet'],
        'strict_ok': strict_pass,
        'strict_reason' : '' if strict_pass else ';'.join(reasons)
    }
    rows.append(row)
    (strict_ok if strict_pass else strict_drop).append(subj)

# Guardar tabla resumen por sujeto
df_summary = pd.DataFrame(rows).sort_values('subject').reset_index(drop=True)
display(df_summary)
df_summary.to_csv(out_table, index=False)
print("Tabla resumen por sujeto guardada en:", out_table)

# Guardar listas Strict
with open(out_ok, 'w', encoding='utf-8') as f:
    for s in strict_ok:
        f.write(f"S{s:03d}\n")
with open(out_drop, 'w', encoding='utf-8') as f:
    for s in strict_drop:
        f.write(f"S{s:03d}\n")
print(f"Strict OK: {len(strict_ok)} sujetos → {out_ok}")
print(f"Strict DROP: {len(strict_drop)} sujetos → {out_drop}")

# Guardar totales globales por clase (retenciones)
df_global = pd.DataFrame(
    [{'y_class': k, 'count': v} for k, v in global_counts.items()]
).sort_values('y_class')
display(df_global)
df_global.to_csv(out_global, index=False)
print("Totales globales por clase guardados en:", out_global)


,subject,epochs_total_possible,epochs_retained,retention_pct,Left,Right,Both Fists,Both Feet,strict_ok,strict_reason
0,1,90,88,97.8,23,21,21,23,True,
1,2,84,84,100.0,21,21,21,21,True,
2,3,90,83,92.2,23,20,20,20,True,
3,4,84,72,85.7,16,17,20,19,True,
4,5,84,84,100.0,21,21,21,21,True,
...,...,...,...,...,...,...,...,...,...,...
98,105,84,84,100.0,21,21,21,21,True,
99,106,90,90,100.0,24,21,23,22,True,
100,107,90,73,81.1,20,14,17,22,True,
101,108,84,62,73.8,15,12,18,17,True,


Tabla resumen por sujeto guardada en: C:\Users\joelc\Desktop\eeg2\reports\tables\02_prepro\subjects_retention_summary.csv
Strict OK: 90 sujetos → C:\Users\joelc\Desktop\eeg2\reports\tables\02_prepro\subjects_strict_OK.txt
Strict DROP: 13 sujetos → C:\Users\joelc\Desktop\eeg2\reports\tables\02_prepro\subjects_strict_DROP.txt


,y_class,count
3,Both Feet,1942
2,Both Fists,1900
0,Left,1939
1,Right,1915


Totales globales por clase guardados en: C:\Users\joelc\Desktop\eeg2\reports\tables\02_prepro\global_class_totals.csv


## Concatenado multi-sujeto

Este bloque reusa los archivos generados por el bloque de retención:

- reports/tables/02_prepro/subjects_strict_OK.txt  → lista de S### admitidos (STRICT)
- data/processed/S###_MI-epo.fif                   → epochs por sujeto ya limpios

Acciones:
1) Concatena TODOS los sujetos disponibles en data/processed → all_subjects-epo.fif
2) Concatena SOLO los sujetos STRICT (del TXT) → strict_subjects-epo.fif

Detalles:
- Reordena los canales de cada sujeto al template del primero.
- Si hay distintas frecuencias de muestreo, re-muestrea al de referencia (del primero).

In [100]:
from glob import glob

# --- rutas de entrada ---
strict_ok_txt = REPORT_TAB / 'subjects_strict_OK.txt'     # generado por tu bloque anterior
epo_glob_all  = str(DATA_PROC / 'S???_MI-epo.fif')        # todos los sujetos limpios

# --- utilidades ---
def _load_epochs_list(file_list):
    """Lee una lista de FIF a memoria (preload=True)."""
    eps = []
    for f in file_list:
        try:
            eps.append(mne.read_epochs(f, preload=True, verbose=False))
        except Exception as e:
            print(f"No pude leer {f}: {e}")
    return eps

def _harmonize(epochs_list):
    """
    Alinea canales y frecuencia:
    - Fija el template de canales al primero.
    - Re-muestrea todos al sfreq del primero (si difieren).
    """
    if not epochs_list:
        return epochs_list
    ref = epochs_list[0]
    ref_chs = ref.ch_names
    ref_fs  = ref.info['sfreq']

    out = [ref]
    for ep in epochs_list[1:]:
        # Reordenar canales si no coinciden exactamente
        if ep.ch_names != ref_chs:
            try:
                ep = ep.copy().reorder_channels(ref_chs)
            except Exception as e:
                # on_mismatch='ignore' también tolera diferencias; intentamos mínima armonización
                print(f"Canales distintos, sigo con on_mismatch='ignore' ({e})")
        # Re-muestrear si fs difiere
        if not np.isclose(ep.info['sfreq'], ref_fs, rtol=0, atol=1e-6):
            ep = ep.copy().resample(ref_fs, npad="auto")
        out.append(ep)
    return out

def _concat_and_save(eps, out_path):
    """Concatena (si hay >=2) y guarda a disco."""
    if len(eps) >= 2:
        # Alinear para evitar errores
        eps = _harmonize(eps)
        epo = mne.concatenate_epochs(eps, on_mismatch='ignore')
        epo.save(out_path, overwrite=True)
        print("Guardado:", out_path, f"(n_epochs={len(epo)})")
    elif len(eps) == 1:
        # Con un solo sujeto, igual guardamos copia por consistencia
        eps[0].save(out_path, overwrite=True)
        print("Guardado (1 sujeto):", out_path, f"(n_epochs={len(eps[0])})")
    else:
        print("No hay sujetos suficientes para concatenar:", out_path.name)

# ========= 1) ALL =========
epo_files_all = sorted(glob(epo_glob_all))
print(f"[ALL] Archivos encontrados: {len(epo_files_all)}")
eps_all = _load_epochs_list(epo_files_all)
_concat_and_save(eps_all, DATA_PROC / 'all_subjects-epo.fif')

# ========= 2) STRICT =========
strict_files = []
if strict_ok_txt.exists():
    strict_ids = []
    with open(strict_ok_txt, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            sid = line.strip()  # ej. "S013"
            if not sid:
                continue
            strict_ids.append(sid)
    # Mapear a FIF existentes
    for sid in strict_ids:
        f = DATA_PROC / f"{sid}_MI-epo.fif"
        if f.exists():
            strict_files.append(str(f))
        else:
            print(f"Falta FIF para {sid}: {f}")
else:
    print("No existe subjects_strict_OK.txt; ejecuta antes el bloque de retención.")

print(f"[STRICT] Archivos encontrados: {len(strict_files)}")
eps_strict = _load_epochs_list(strict_files)
_concat_and_save(eps_strict, DATA_PROC / 'strict_subjects-epo.fif')

print(f"Resumen → ALL: {len(eps_all)} sujetos, STRICT: {len(eps_strict)} sujetos")


[ALL] Archivos encontrados: 103
Guardado: C:\Users\joelc\Desktop\eeg2\data\processed\all_subjects-epo.fif (n_epochs=7696)
[STRICT] Archivos encontrados: 90
Guardado: C:\Users\joelc\Desktop\eeg2\data\processed\strict_subjects-epo.fif (n_epochs=7276)
Resumen → ALL: 103 sujetos, STRICT: 90 sujetos
